In [1]:
from csgo.parser import DemoParser
from csgo.analytics import stats
import numpy
import pandas as pd
import os

In [2]:
def getDemoPaths(directory):
    demoPaths = []
    for filename in os.listdir(directory):
        demoPaths.append(os.path.join(directory, filename))
    return demoPaths


In [3]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [4]:
def generateTotalData(parsers):
    totalData = []
    for x in parsers:
        totalData.append(x.parse())
    return totalData

In [5]:
def generateData(parsers):
    totalData, killData, roundData, damageData, frameData, playerFrames, bombEvents, grenadeData, flashData = [], [], [], [], [], [], [], [], []
    for x in parsers:
        totalData.append(x.parse())
        killData.append(x._parse_kills(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
        frameData.append(x._parse_frames(return_type="df"))
        playerFrames.append(x._parse_player_frames(return_type="df"))
        bombEvents.append(x._parse_bomb_events(return_type="df"))
        grenadeData.append(x._parse_grenades(return_type="df"))
        flashData.append(x._parse_flashes(return_type="df"))
    return (totalData, killData, roundData, damageData, frameData, playerFrames, bombEvents, grenadeData, flashData)

In [6]:
def createVsMatrix():
    return {'5' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}},
                      '4' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}}, 
                      '3' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}},
                      '2' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}},
                      '1' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}}}

In [7]:
def createCT_T_VS_Breakdowns(totalData, ct_breakdown, t_breakdown, team):
    for x in totalData:
        for y in x['GameRounds']:
            winningTeam = (y['WinningTeam'] == team)
            for z in y['Frames']:
                if(z['T']['TeamName'] == team):
                    if(z['T']['AlivePlayers'] != 0 and z['CT']['AlivePlayers'] != 0):
                        if(not t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['changed']):
                            t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['Total'] += 1
                            if(winningTeam):
                                t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['Win'] += 1
                            else:
                                t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['Lose'] += 1
                            t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['changed'] = True
                elif(z['CT']['TeamName'] == team):
                    if(z['T']['AlivePlayers'] != 0 and z['CT']['AlivePlayers'] != 0):
                        if(not ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['changed']):
                            ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['Total'] += 1
                            if(winningTeam):
                                ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['Win'] += 1
                            else:
                                ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['Lose'] += 1
                            ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['changed'] = True
            for i in ct_breakdown:
                for j in ct_breakdown[i]:
                    ct_breakdown[i][j]['changed'] = False
            for i in t_breakdown:
                for j in t_breakdown[i]:
                    t_breakdown[i][j]['changed'] = False

In [8]:
def generateWr(side_breakdown):
    side_wrs = {}
    for x in side_breakdown:
        for y in side_breakdown[x]:
            side_wrs[str(x) + " vs " + str(y)] = side_breakdown[x][y]['Win'] / side_breakdown[x][y]['Total']
    return side_wrs

In [9]:
nitro_path = "/mnt/d/Team Liquid/Nitro Lineup/"
taco_path = "/mnt/d/Team Liquid/Taco Lineup/"
fallen_path = "/mnt/d/Team Liquid/Fallen Lineup/"

In [11]:
fTotalData = generateTotalData(generateDemoParsers(getDemoPaths(fallen_path)))
nTotalData = generateTotalData(generateDemoParsers(getDemoPaths(nitro_path)))
ttotalData = generateTotalData(generateDemoParsers(getDemoPaths(taco_path)))

16:44:06 [INFO] Go version>=1.14.0
16:44:06 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Team Liquid/Fallen Lineup/astralis-vs-liquid-m1-overpass.dem
16:44:06 [INFO] Setting demo id to astralis-vs-liquid-m1-overpass
16:44:06 [INFO] Setting parse rate to 128
16:44:07 [INFO] Go version>=1.14.0
16:44:07 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Team Liquid/Fallen Lineup/astralis-vs-liquid-m2-dust2.dem
16:44:07 [INFO] Setting demo id to astralis-vs-liquid-m2-dust2
16:44:07 [INFO] Setting parse rate to 128
16:44:08 [INFO] Go version>=1.14.0
16:44:08 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Team Liquid/Fallen Lineup/endpoint-vs-liquid-m1-inferno.dem
16:44:08 [INFO] Setting demo id to endpoint-vs-liquid-m1-inferno
16:44:08 [INFO] Setting parse rate to 128
16:44:08 [INFO] Go version>=1.14.0
16:44:08 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Team Liquid/Fallen Lineup/endpoint-vs-liquid-m2-vertigo.dem
16:44:08 [INFO] Setting demo id to endpoint-v

16:44:26 [INFO] Setting demo id to liquid-vs-mousesports-m3-inferno
16:44:26 [INFO] Setting parse rate to 128
16:44:26 [INFO] Go version>=1.14.0
16:44:26 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Team Liquid/Fallen Lineup/liquid-vs-nip-m1-overpass.dem
16:44:26 [INFO] Setting demo id to liquid-vs-nip-m1-overpass
16:44:26 [INFO] Setting parse rate to 128
16:44:27 [INFO] Go version>=1.14.0
16:44:27 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Team Liquid/Fallen Lineup/liquid-vs-nip-m2-dust2.dem
16:44:27 [INFO] Setting demo id to liquid-vs-nip-m2-dust2
16:44:27 [INFO] Setting parse rate to 128
16:44:28 [INFO] Go version>=1.14.0
16:44:28 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Team Liquid/Fallen Lineup/liquid-vs-nip-m3-mirage.dem
16:44:28 [INFO] Setting demo id to liquid-vs-nip-m3-mirage
16:44:28 [INFO] Setting parse rate to 128
16:44:28 [INFO] Go version>=1.14.0
16:44:28 [INFO] Initialized CSGODemoParser with demofile /mnt/d/Team Liquid/Fallen Lineup/l

# Demo Parsers: 42


16:45:08 [INFO] Wrote demo parse output to astralis-vs-liquid-m1-overpass.json
16:45:08 [INFO] Reading in JSON from astralis-vs-liquid-m1-overpass.json
16:45:09 [INFO] JSON data loaded, available in the `json` attribute to parser
16:45:09 [INFO] Successfully parsed JSON output
16:45:09 [INFO] Successfully returned JSON output
16:45:09 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:45:09 [INFO] Looking for file at /mnt/d/Team Liquid/Fallen Lineup/astralis-vs-liquid-m2-dust2.dem
16:45:30 [INFO] Wrote demo parse output to astralis-vs-liquid-m2-dust2.json
16:45:30 [INFO] Reading in JSON from astralis-vs-liquid-m2-dust2.json
16:45:31 [INFO] JSON data loaded, available in the `json` attribute to parser
16:45:31 [INFO] Successfully parsed JSON output
16:45:31 [INFO] Successfully returned JSON output
16:45:31 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:45:31 [INFO] Looking for file at /mnt/d/Team Liquid/Fallen Lineup/endpoint-vs-li

16:50:26 [INFO] JSON data loaded, available in the `json` attribute to parser
16:50:26 [INFO] Successfully parsed JSON output
16:50:26 [INFO] Successfully returned JSON output
16:50:26 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:50:26 [INFO] Looking for file at /mnt/d/Team Liquid/Fallen Lineup/gambit-vs-liquid-m2-vertigo.dem
16:50:47 [INFO] Wrote demo parse output to gambit-vs-liquid-m2-vertigo.json
16:50:47 [INFO] Reading in JSON from gambit-vs-liquid-m2-vertigo.json
16:50:48 [INFO] JSON data loaded, available in the `json` attribute to parser
16:50:48 [INFO] Successfully parsed JSON output
16:50:48 [INFO] Successfully returned JSON output
16:50:48 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:50:48 [INFO] Looking for file at /mnt/d/Team Liquid/Fallen Lineup/gambit-vs-liquid-m3-mirage.dem
16:51:03 [INFO] Wrote demo parse output to gambit-vs-liquid-m3-mirage.json
16:51:03 [INFO] Reading in JSON from gambit-vs-liquid-m3-mir

...

17:16:03 [INFO] Looking for file at /mnt/d/Team Liquid/Taco Lineup/astralis-vs-liquid-inferno_2323561.dem


# Demo Parsers: 47


17:16:11 [INFO] Wrote demo parse output to astralis-vs-liquid-inferno.json
17:16:11 [INFO] Reading in JSON from astralis-vs-liquid-inferno.json
17:16:12 [INFO] JSON data loaded, available in the `json` attribute to parser
17:16:12 [INFO] Successfully parsed JSON output
17:16:12 [INFO] Successfully returned JSON output
17:16:12 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
17:16:12 [INFO] Looking for file at /mnt/d/Team Liquid/Taco Lineup/astralis-vs-liquid-inferno_2326404.dem
17:16:34 [INFO] Wrote demo parse output to astralis-vs-liquid-inferno.json
17:16:34 [INFO] Reading in JSON from astralis-vs-liquid-inferno.json
17:16:35 [INFO] JSON data loaded, available in the `json` attribute to parser
17:16:35 [INFO] Successfully parsed JSON output
17:16:35 [INFO] Successfully returned JSON output
17:16:35 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
17:16:35 [INFO] Looking for file at /mnt/d/Team Liquid/Taco Lineup/avangar-vs-liquid-m1-

17:20:06 [INFO] JSON data loaded, available in the `json` attribute to parser
17:20:06 [INFO] Successfully parsed JSON output
17:20:06 [INFO] Successfully returned JSON output
17:20:06 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
17:20:06 [INFO] Looking for file at /mnt/d/Team Liquid/Taco Lineup/liquid-vs-astralis-m1-mirage_2323569.dem
17:20:18 [INFO] Wrote demo parse output to liquid-vs-astralis-m1-mirage.json
17:20:18 [INFO] Reading in JSON from liquid-vs-astralis-m1-mirage.json
17:20:18 [INFO] JSON data loaded, available in the `json` attribute to parser
17:20:18 [INFO] Successfully parsed JSON output
17:20:18 [INFO] Successfully returned JSON output
17:20:18 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
17:20:18 [INFO] Looking for file at /mnt/d/Team Liquid/Taco Lineup/liquid-vs-astralis-m1-nuke_2322814.dem
17:20:37 [INFO] Wrote demo parse output to liquid-vs-astralis-m1-nuke.json
17:20:37 [INFO] Reading in JSON from liquid-v

17:25:31 [INFO] JSON data loaded, available in the `json` attribute to parser
17:25:31 [INFO] Successfully parsed JSON output
17:25:31 [INFO] Successfully returned JSON output
17:25:31 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
17:25:31 [INFO] Looking for file at /mnt/d/Team Liquid/Taco Lineup/liquid-vs-mousesports-m2-train_2323786.dem
17:25:41 [INFO] Wrote demo parse output to liquid-vs-mousesports-m2-train.json
17:25:41 [INFO] Reading in JSON from liquid-vs-mousesports-m2-train.json
17:25:42 [INFO] JSON data loaded, available in the `json` attribute to parser
17:25:42 [INFO] Successfully parsed JSON output
17:25:42 [INFO] Successfully returned JSON output
17:25:42 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
17:25:42 [INFO] Looking for file at /mnt/d/Team Liquid/Taco Lineup/liquid-vs-nrg-m1-cache_2323113.dem
17:25:57 [INFO] Wrote demo parse output to liquid-vs-nrg-m1-cache.json
17:25:57 [INFO] Reading in JSON from liquid-vs-

In [ ]:
f_ct_breakdown =  createVsMatrix()
f_t_breakdown = createVsMatrix()

n_ct_breakdown =  createVsMatrix()
n_t_breakdown = createVsMatrix()

t_ct_breakdown =  createVsMatrix()
t_t_breakdown = createVsMatrix()

In [ ]:
createCT_T_VS_Breakdowns(fTotalData, f_ct_breakdown, f_t_breakdown, 'Team Liquid')
createCT_T_VS_Breakdowns(nTotalData, n_ct_breakdown, n_t_breakdown, 'Team Liquid')
createCT_T_VS_Breakdowns(ttotalData, t_ct_breakdown, t_t_breakdown, 'Team Liquid')

In [ ]:
f_ct_wr = generateWr(f_ct_breakdown)
f_t_wr = generateWr(f_t_breakdown)

n_ct_wr = generateWr(n_ct_breakdown)
n_t_wr = generateWr(n_t_breakdown)

t_ct_wr = generateWr(t_ct_breakdown)
t_t_wr = generateWr(t_t_breakdown)

In [ ]:
nitro = []
for x in n_t_wr:
    nitro.append(n_t_wr[x])

In [ ]:
fallen = []
for x in f_t_wr:
    fallen.append(f_t_wr[x])

In [ ]:
taco = []
for x in t_t_wr:
    taco.append(t_t_wr[x])

In [ ]:
nitroct = []
for x in n_ct_wr:
    nitro.append(n_ct_wr[x])

In [ ]:
fallenct = []
for x in f_ct_wr:
    fallen.append(f_ct_wr[x])

In [ ]:
tacoct = []
for x in t_ct_wr:
    taco.append(t_ct_wr[x])

In [ ]:
df_to_export_ct = pd.DataFrame(list(zip(n_ct_wr.keys(), nitroct, fallenct, tacoct)), columns = ['X vs X Situation', 'Nitro Lineup', 'Fallen Lineup', 'Taco Lineup'])

In [ ]:
df_to_export = pd.DataFrame(list(zip(n_t_wr.keys(), nitro, fallen, taco)), columns = ['X vs X Situation', 'Nitro Lineup', 'Fallen Lineup', 'Taco Lineup'])

In [ ]:
df_to_export.set_index('X vs X Situation')

In [ ]:
df_to_export_ct.set_index('X vs X Situation')

In [ ]:
df_to_export.to_csv("TL_T_Win_rates.csv")

In [ ]:
df_to_export_ct.to_csv("TL_CT_Win_rates.csv")

In [ ]:
f_ct_wr